In [49]:
import numpy as np
import pandas as pd
import torch
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [24]:
import csv
wine_path = "https://raw.githubusercontent.com/duybluemind1988/dlwpt-code/master/data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [55]:
col_list=list(pd.read_csv(wine_path,delimiter=";").columns)
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [56]:
#col_list = next(csv.reader(open(wine_path), delimiter=';'))
#col_list

In [25]:
#col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, #col_list

((4898, 12),)

In [26]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [27]:
data = wineq[:, :-1] # <1>
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]), torch.Size([4898, 11]))

In [28]:
target = wineq[:, -1] # <2>
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [29]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

In [30]:
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [31]:
target.unsqueeze(1)

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

We can achieve one-hot encoding using the scatter_ method, which fills the tensor with values from a source tensor along the indices provided as arguments

Let’s see what scatter_ does. First, we notice that its name ends with an underscore.
As you learned in the previous chapter, this is a convention in PyTorch that indicates
the method will not return a new tensor, but will instead modify the tensor in place.
The arguments for scatter_ are as follows:

 The dimension along which the following two arguments are specified
 A column tensor indicating the indices of the elements to scatter
 A tensor containing the elements to scatter or a single scalar to scatter (1, in
this case)

In other words, the previous invocation reads, “For each row, take the index of the target label (which coincides with the score in our case) and use it as the column index
to set the value 1.0.” The end result is a tensor encoding categorical information


In [32]:
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

The call to unsqueeze adds a singleton dimension, from a 1D tensor of 4,898 elements
to a 2D tensor of size (4,898 × 1), without changing its contents—no extra elements
are added

In [33]:
target

tensor([6, 6,  ..., 7, 6])

In [34]:
target[0]

tensor(6)

In [35]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [36]:
target_unsqueezed[0][0]

tensor(6)

In [18]:
data

tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
        [ 6.30,  0.30,  ...,  0.49,  9.50],
        ...,
        [ 5.50,  0.29,  ...,  0.38, 12.80],
        [ 6.00,  0.21,  ...,  0.32, 11.80]])

In [38]:
data.shape

torch.Size([4898, 11])

In [19]:
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.85e+00, 2.78e-01, 3.34e-01, 6.39e+00, 4.58e-02, 3.53e+01,
        1.38e+02, 9.94e-01, 3.19e+00, 4.90e-01, 1.05e+01])

In [37]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
        1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00])

In [39]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.72e-01, -8.18e-02,  ..., -3.49e-01, -1.39e+00],
        [-6.57e-01,  2.16e-01,  ...,  1.35e-03, -8.24e-01],
        ...,
        [-1.61e+00,  1.17e-01,  ..., -9.63e-01,  1.86e+00],
        [-1.01e+00, -6.77e-01,  ..., -1.49e+00,  1.04e+00]])

In [40]:
bad_indexes = target <= 3 # <1>
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [41]:
bad_indexes

tensor([False, False,  ..., False, False])

In [42]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [43]:
bad_data

tensor([[8.50e+00, 2.60e-01, 2.10e-01, 1.62e+01, 7.40e-02, 4.10e+01,
         1.97e+02, 9.98e-01, 3.02e+00, 5.00e-01, 9.80e+00],
        [5.80e+00, 2.40e-01, 4.40e-01, 3.50e+00, 2.90e-02, 5.00e+00,
         1.09e+02, 9.91e-01, 3.53e+00, 4.30e-01, 1.17e+01],
        [9.10e+00, 5.90e-01, 3.80e-01, 1.60e+00, 6.60e-02, 3.40e+01,
         1.82e+02, 9.97e-01, 3.23e+00, 3.80e-01, 8.50e+00],
        [7.10e+00, 3.20e-01, 3.20e-01, 1.10e+01, 3.80e-02, 1.60e+01,
         6.60e+01, 9.94e-01, 3.24e+00, 4.00e-01, 1.15e+01],
        [6.90e+00, 3.90e-01, 4.00e-01, 4.60e+00, 2.20e-02, 5.00e+00,
         1.90e+01, 9.92e-01, 3.31e+00, 3.70e-01, 1.26e+01],
        [1.03e+01, 1.70e-01, 4.70e-01, 1.40e+00, 3.70e-02, 5.00e+00,
         3.30e+01, 9.94e-01, 2.89e+00, 2.80e-01, 9.60e+00],
        [7.90e+00, 6.40e-01, 4.60e-01, 1.06e+01, 2.44e-01, 3.30e+01,
         2.27e+02, 9.98e-01, 2.87e+00, 7.40e-01, 9.10e+00],
        [8.30e+00, 3.30e-01, 4.20e-01, 1.15e+00, 3.30e-02, 1.80e+01,
         9.60e+01, 9.91e-01,

In [57]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)] # <1>
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [62]:
bad_mean

tensor([7.60e+00, 3.33e-01, 3.36e-01, 6.39e+00, 5.43e-02, 5.33e+01,
        1.71e+02, 9.95e-01, 3.19e+00, 4.75e-01, 1.03e+01])

In [59]:
mid_mean

tensor([6.89e+00, 2.82e-01, 3.36e-01, 6.71e+00, 4.78e-02, 3.54e+01,
        1.42e+02, 9.94e-01, 3.18e+00, 4.87e-01, 1.03e+01])

In [61]:
good_mean

tensor([6.73e+00, 2.65e-01, 3.26e-01, 5.26e+00, 3.82e-02, 3.46e+01,
        1.25e+02, 9.92e-01, 3.22e+00, 5.00e-01, 1.14e+01])

In [58]:
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [60]:
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [64]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
total_sulfur_data

tensor([170., 132.,  ..., 110.,  98.])

In [65]:
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes

tensor([False,  True,  ...,  True,  True])

In [66]:
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [67]:
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [68]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)